<a href="https://colab.research.google.com/github/bpopeters/transducer-tutorial/blob/main/transducer_tutorial_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transducer implementation in PyTorch

*by Loren Lugosch*



In this notebook, we will implement a Transducer sequence-to-sequence model for inserting missing vowels into a sentence ("Hll, Wrld" --> "Hello, World").

In [18]:
import torch
import string
import numpy as np
import itertools
from collections import Counter
from tqdm import tqdm
!pip install unidecode
import unidecode


# Building blocks

First, we will define the encoder, predictor, and joiner using standard neural nets.

<img src="https://lorenlugosch.github.io/images/transducer/transducer-model.png" width="25%">

In [19]:
NULL_INDEX = 0

encoder_dim = 128
predictor_dim = 128
joiner_dim = 128

The encoder is any network that can take as input a variable-length sequence: so, RNNs, CNNs, and self-attention/Transformer encoders will all work.


In [20]:
class Encoder(torch.nn.Module):
  def __init__(self, num_inputs):
    super(Encoder, self).__init__()
    self.embed = torch.nn.Embedding(num_inputs, encoder_dim)
    self.rnn = torch.nn.GRU(input_size=encoder_dim, hidden_size=encoder_dim, num_layers=3, batch_first=True, bidirectional=True, dropout=0.1)
    self.linear = torch.nn.Linear(encoder_dim*2, joiner_dim)

  def forward(self, x):
    out = x
    out = self.embed(out)
    out = self.rnn(out)[0]
    out = self.linear(out)
    return out

The predictor is any _causal_ network (= can't look at the future): in other words, unidirectional RNNs, causal convolutions, or masked self-attention. 

In [21]:
class Predictor(torch.nn.Module):
  def __init__(self, num_outputs):
    super(Predictor, self).__init__()
    self.embed = torch.nn.Embedding(num_outputs, predictor_dim)
    self.rnn = torch.nn.GRUCell(input_size=predictor_dim, hidden_size=predictor_dim)
    self.linear = torch.nn.Linear(predictor_dim, joiner_dim)
    
    self.initial_state = torch.nn.Parameter(torch.randn(predictor_dim))
    self.start_symbol = NULL_INDEX # In the original paper, a vector of 0s is used; just using the null index instead is easier when using an Embedding layer.

  def forward_one_step(self, input, previous_state):
    embedding = self.embed(input)
    state = self.rnn.forward(embedding, previous_state)
    out = self.linear(state)
    return out, state

  def forward(self, y):
    batch_size = y.shape[0]
    U = y.shape[1]
    outs = []
    state = torch.stack([self.initial_state] * batch_size).to(y.device)
    for u in range(U+1): # need U+1 to get null output for final timestep 
      if u == 0:
        decoder_input = torch.tensor([self.start_symbol] * batch_size).to(y.device)
      else:
        decoder_input = y[:,u-1]
      out, state = self.forward_one_step(decoder_input, state)
      outs.append(out)
    out = torch.stack(outs, dim=1)
    return out

The joiner is a feedforward network/MLP with one hidden layer applied independently to each $(t,u)$ index.

(The linear part of the hidden layer is contained in the encoder and predictor, so we just do the nonlinearity here and then the output layer.)

In [22]:
class Joiner(torch.nn.Module):
  def __init__(self, num_outputs):
    super(Joiner, self).__init__()
    self.linear = torch.nn.Linear(joiner_dim, num_outputs)

  def forward(self, encoder_out, predictor_out):
    out = encoder_out + predictor_out
    out = torch.nn.functional.relu(out)
    out = self.linear(out)
    return out

# Transducer model + loss function

Using the encoder, predictor, and joiner, we will implement the Transducer model and its loss function.

<img src="https://lorenlugosch.github.io/images/transducer/forward-messages.png" width="25%">

We can use a simple PyTorch implementation of the loss function, relying on automatic differentiation to give us gradients.

In [23]:
class Transducer(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super(Transducer, self).__init__()
    self.encoder = Encoder(num_inputs)
    self.predictor = Predictor(num_outputs)
    self.joiner = Joiner(num_outputs)

    if torch.cuda.is_available(): self.device = "cuda:0"
    else: self.device = "cpu"
    self.to(self.device)

  def compute_forward_prob(self, joiner_out, T, U, y):
    """
    joiner_out: tensor of shape (B, T_max, U_max+1, #labels)
    T: list of input lengths
    U: list of output lengths 
    y: label tensor (B, U_max+1)
    """
    B = joiner_out.shape[0]
    T_max = joiner_out.shape[1]
    U_max = joiner_out.shape[2] - 1
    log_alpha = torch.zeros(B, T_max, U_max+1).to(model.device)
    for t in range(T_max):
      for u in range(U_max+1):
          if u == 0:
            if t == 0:
              log_alpha[:, t, u] = 0.

            else: #t > 0
              log_alpha[:, t, u] = log_alpha[:, t-1, u] + joiner_out[:, t-1, 0, NULL_INDEX] 
                  
          else: #u > 0
            if t == 0:
              log_alpha[:, t, u] = log_alpha[:, t,u-1] + torch.gather(joiner_out[:, t, u-1], dim=1, index=y[:,u-1].view(-1,1) ).reshape(-1)
            
            else: #t > 0
              log_alpha[:, t, u] = torch.logsumexp(torch.stack([
                  log_alpha[:, t-1, u] + joiner_out[:, t-1, u, NULL_INDEX],
                  log_alpha[:, t, u-1] + torch.gather(joiner_out[:, t, u-1], dim=1, index=y[:,u-1].view(-1,1) ).reshape(-1)
              ]), dim=0)
    
    log_probs = []
    for b in range(B):
      log_prob = log_alpha[b, T[b]-1, U[b]] + joiner_out[b, T[b]-1, U[b], NULL_INDEX]
      log_probs.append(log_prob)
    log_probs = torch.stack(log_probs) 
    return log_prob

  def compute_loss(self, x, y, T, U):
    encoder_out = self.encoder.forward(x)
    predictor_out = self.predictor.forward(y)
    joiner_out = self.joiner.forward(encoder_out.unsqueeze(2), predictor_out.unsqueeze(1)).log_softmax(3)
    loss = -self.compute_forward_prob(joiner_out, T, U, y).mean()
    return loss

Let's first verify that the forward algorithm actually correctly computes the sum (in log space, the [logsumexp](https://lorenlugosch.github.io/posts/2020/06/logsumexp/)) of all possible alignments, using a short input/output pair for which computing all possible alignments is feasible.

<img src="https://lorenlugosch.github.io/images/transducer/cat-align-1.png" width="25%">

In [24]:
def compute_single_alignment_prob(self, encoder_out, predictor_out, T, U, z, y):
    """
    Computes the probability of one alignment, z.
    """
    t = 0; u = 0
    t_u_indices = []
    y_expanded = []
    for step in z:
      t_u_indices.append((t,u))
      if step == 0: # right (null)
        y_expanded.append(NULL_INDEX)
        t += 1
      if step == 1: # down (label)
        y_expanded.append(y[u])
        u += 1
    t_u_indices.append((T-1,U))
    y_expanded.append(NULL_INDEX)

    t_indices = [t for (t,u) in t_u_indices]
    u_indices = [u for (t,u) in t_u_indices]
    encoder_out_expanded = encoder_out[t_indices]
    predictor_out_expanded = predictor_out[u_indices]
    joiner_out = self.joiner.forward(encoder_out_expanded, predictor_out_expanded).log_softmax(1)
    logprob = -torch.nn.functional.nll_loss(input=joiner_out, target=torch.tensor(y_expanded).long().to(self.device), reduction="sum")
    return logprob

Transducer.compute_single_alignment_prob = compute_single_alignment_prob

In [25]:
# Generate example inputs/outputs
num_outputs = len(string.ascii_uppercase) + 1 # [null, A, B, ... Z]
model = Transducer(1, num_outputs)
y_letters = "CAT"
y = torch.tensor([string.ascii_uppercase.index(l) + 1 for l in y_letters]).unsqueeze(0).to(model.device)
T = torch.tensor([4]); U = torch.tensor([len(y_letters)]); B = 1

encoder_out = torch.randn(B, T, joiner_dim).to(model.device)
predictor_out = torch.randn(B, U+1, joiner_dim).to(model.device)
joiner_out = model.joiner.forward(encoder_out.unsqueeze(2), predictor_out.unsqueeze(1)).log_softmax(3)

#######################################################
# Compute loss by enumerating all possible alignments #
#######################################################
all_permutations = list(itertools.permutations([0]*(T-1) + [1]*U))
all_distinct_permutations = list(Counter(all_permutations).keys())
alignment_probs = []
for z in all_distinct_permutations:
  alignment_prob = model.compute_single_alignment_prob(encoder_out[0], predictor_out[0], T.item(), U.item(), z, y[0])
  alignment_probs.append(alignment_prob)
loss_enumerate = -torch.tensor(alignment_probs).logsumexp(0)

#######################################################
# Compute loss using the forward algorithm            #
#######################################################
loss_forward = -model.compute_forward_prob(joiner_out, T, U, y)

print("Loss computed by enumerating all possible alignments: ", loss_enumerate)
print("Loss computed using the forward algorithm: ", loss_forward)

Loss computed by enumerating all possible alignments:  tensor(19.3340)
Loss computed using the forward algorithm:  tensor(19.3340, device='cuda:0', grad_fn=<NegBackward0>)


Now let's add the greedy search algorithm for predicting an output sequence.

(Note that I've assumed we're using RNNs for the predictor here. You would have to modify this code a bit if you want to use convolutions/self-attention instead.) 
<br/><br/>
<img src="https://lorenlugosch.github.io/images/transducer/greedy-search.png" width="50%">

In [26]:
def greedy_search(self, x, T):
  y_batch = []
  B = len(x)
  encoder_out = self.encoder.forward(x)
  U_max = 200
  for b in range(B):
    t = 0; u = 0; y = [self.predictor.start_symbol]; predictor_state = self.predictor.initial_state.unsqueeze(0)
    while t < T[b] and u < U_max:
      predictor_input = torch.tensor([ y[-1] ]).to(x.device)
      g_u, predictor_state = self.predictor.forward_one_step(predictor_input, predictor_state)
      f_t = encoder_out[b, t]
      h_t_u = self.joiner.forward(f_t, g_u)
      argmax = h_t_u.max(-1)[1].item()
      if argmax == NULL_INDEX:
        t += 1
      else: # argmax == a label
        u += 1
        y.append(argmax)
    y_batch.append(y[1:]) # remove start symbol
  return y_batch

Transducer.greedy_search = greedy_search

The code above will work, but training will be very slow because the Transducer loss is written in pure Python. You can use the fast implementation from SpeechBrain instead by running the block below.

In [27]:
!pip install speechbrain
from speechbrain.nnet.loss.transducer_loss import TransducerLoss
transducer_loss = TransducerLoss(0)

def compute_loss(self, x, y, T, U):
    encoder_out = self.encoder.forward(x)
    predictor_out = self.predictor.forward(y)
    joiner_out = self.joiner.forward(encoder_out.unsqueeze(2), predictor_out.unsqueeze(1)).log_softmax(3)
    #loss = -self.compute_forward_prob(joiner_out, T, U, y).mean()
    T = T.to(joiner_out.device)
    U = U.to(joiner_out.device)
    loss = transducer_loss(joiner_out, y, T, U) #, blank_index=NULL_INDEX, reduction="mean")
    return loss

Transducer.compute_loss = compute_loss

# Some utilities

Here we will add a bit of boilerplate code for training and loading data.

In [36]:
def read_corpus(path):
  with open(path) as f:
    return [line.strip().split("\t") for line in f if line != "\n"]

train = read_corpus("hun_train.tsv")
dev = read_corpus("hun_dev.tsv")
test = read_corpus("hun_test.tsv")

src, tgt = zip(*train)
src_itos = ["_"] + list(set(itertools.chain.from_iterable([list(s) for s in src])))
tgt_itos = ["_"] + list(set(itertools.chain.from_iterable([t.split() for t in tgt])))
print(src_itos)
print(tgt_itos)

src_stoi = {s: i for i, s in enumerate(src_itos)}
tgt_stoi = {t: i for i, t in enumerate(tgt_itos)}

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, lines, batch_size):

    self.lines = lines # list of pairs of strings
    collate = Collate()
    self.loader = torch.utils.data.DataLoader(self, batch_size=batch_size, num_workers=1, shuffle=True, collate_fn=collate)

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx):
    x, y = self.lines[idx]
    return x, y

# encode and decode functions will need to be different
# encode turns strings into lists of indices
def encode_src(s):
  return [src_stoi[c] for c in s]

def encode_tgt(t):
  return [tgt_stoi[c] for c in t.split()]

def decode_src(seq):
  return "".join([src_itos[i] for i in seq])

def decode_tgt(seq):
  return " ".join([tgt_itos[i] for i in seq])

class Collate:
  def __call__(self, batch):
    """
    batch: list of tuples (input string, output string)
    Returns a minibatch of strings, encoded as labels and padded to have the same length.
    """
    x = []; y = []
    batch_size = len(batch)
    for index in range(batch_size):
      x_, y_ = batch[index]
      x.append(encode_src(x_))
      y.append(encode_tgt(y_))

    # pad all sequences to have same length
    T = [len(x_) for x_ in x]
    U = [len(y_) for y_ in y]
    T_max = max(T)
    U_max = max(U)
    for index in range(batch_size):
      x[index] += [NULL_INDEX] * (T_max - len(x[index]))
      x[index] = torch.tensor(x[index])
      y[index] += [NULL_INDEX] * (U_max - len(y[index]))
      y[index] = torch.tensor(y[index])

    # stack into single tensor
    x = torch.stack(x)
    y = torch.stack(y)
    T = torch.tensor(T)
    U = torch.tensor(U)

    return (x,y,T,U)

train_set = TextDataset(train, batch_size=64) #8)
dev_set = TextDataset(dev, batch_size=64)
test_set = TextDataset(test, batch_size=64) #8)
train_set.__getitem__(0)

['_', 'l', 'ő', 'v', 'u', 'ö', 'ú', 'n', 'g', 'ű', 'm', 't', 'p', 'é', 'f', 'c', 'd', 'i', 'y', 'o', 'k', 'j', 'á', 's', 'h', 'x', 'ü', 'w', 'z', 'ó', 'e', 'í', 'a', 'b', 'r']
['_', 'tː', 'l', 'ʒ', 'ɲː', 'v', 'u', 'ɦ', 'dː', 'zː', 'n', 'iː', 't͡ʃː', 'jː', 'ɱ', 't͡ʃ', 'ŋ', 'vː', 'mː', 'aː', 'lː', 'p', 't', 'ɒ', 'm', 'ɲ', 'fː', 'ɟ', 'cː', 'ø', 'f', 'yː', 'ʃː', 'kː', 'c', 'bː', 'd', 'ɟː', 'd͡ʒː', 'i', 'sː', 'y', 'o', 'k', 't͡s', 'rː', 'j', 'ɛ', 'ɡ', 's', 'ʝ', 'h', 't͡sː', 'x', 'z', 'd͡ʒ', 'øː', 'uː', 'eː', 'ʃ', 'oː', 'b', 'r', 'ɡː', 'nː', 'd͡zː', 'pː']


('abba', 'ɒ bː ɒ')

In [34]:
class Trainer:
  def __init__(self, model, lr):
    self.model = model
    self.lr = lr
    self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr)
  
  def train(self, dataset, print_interval = 20):
    train_loss = 0
    num_samples = 0
    self.model.train()
    pbar = tqdm(dataset.loader)
    for idx, batch in enumerate(pbar):
      x,y,T,U = batch
      x = x.to(self.model.device); y = y.to(self.model.device)
      batch_size = len(x)
      num_samples += batch_size
      loss = self.model.compute_loss(x,y,T,U)
      self.optimizer.zero_grad()
      pbar.set_description("%.2f" % loss.item())
      loss.backward()
      self.optimizer.step()
      train_loss += loss.item() * batch_size
      if idx % print_interval == 0:
        self.model.eval()
        guesses = self.model.greedy_search(x,T)
        self.model.train()
        print("\n")
        for b in range(2):
          print("input:", decode_src(x[b,:T[b]]))
          print("guess:", decode_tgt(guesses[b]))
          print("truth:", decode_tgt(y[b,:U[b]]))
          print("")
    train_loss /= num_samples
    return train_loss

  def test(self, dataset, print_interval=1):
    test_loss = 0
    num_samples = 0
    self.model.eval()
    pbar = tqdm(dataset.loader)
    for idx, batch in enumerate(pbar):
      x,y,T,U = batch
      x = x.to(self.model.device); y = y.to(self.model.device)
      batch_size = len(x)
      num_samples += batch_size
      loss = self.model.compute_loss(x,y,T,U)
      pbar.set_description("%.2f" % loss.item())
      test_loss += loss.item() * batch_size
      if idx % print_interval == 0:
        print("\n")
        print("input:", decode_src(x[0,:T[0]]))
        print("\nguess:", decode_tgt(self.model.greedy_search(x,T)[0]))
        print("truth:", decode_tgt(y[0,:U[0]]))
        print("")
    test_loss /= num_samples
    return test_loss
    

# Training the model

Now we will train a model. This will generate some output sequences every 20 batches.

In [38]:
model = Transducer(num_inputs=len(src_itos), num_outputs=len(tgt_itos))
trainer = Trainer(model=model, lr=0.0003)

num_epochs = 20
train_losses=[]
test_losses=[]

for epoch in range(num_epochs):
    train_loss = trainer.train(train_set)
    dev_loss = trainer.test(dev_set)
    train_losses.append(train_loss)
    test_losses.append(dev_loss)
    print("Epoch %d: train loss = %f, test loss = %f" % (epoch, train_loss, dev_loss))

6.53:   3%|▎         | 4/125 [00:04<01:45,  1.15it/s]



input: klasszikus
guess: i oː f uː t u ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f ʃː ʃː f
truth: k l ɒ sː i k u ʃ

input: eredetű
guess: f i f uː t u ʃː f ʃː f f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f ʃː f uː uː ɲ f 

3.96:  20%|██        | 25/125 [00:05<00:07, 12.92it/s]



input: létesítmény
guess: 
truth: l eː t ɛ ʃ iː t m eː ɲ

input: rovarok
guess: 
truth: r o v ɒ r o k



3.47:  38%|███▊      | 47/125 [00:06<00:03, 20.94it/s]

...

guess: p r oː b ɒ
truth: p r oː b ɒ



input: dolgok


0.01:  94%|█████████▍| 15/16 [00:05<00:00,  2.82it/s]

guess: d o l o k
truth: d o l ɡ o k



input: jóban


0.01: 100%|██████████| 16/16 [00:05<00:00,  2.77it/s]

guess: j oː b ɒ n
truth: j oː b ɒ n

Epoch 19: train loss = 0.011076, test loss = 0.040175


In [39]:
print(train_losses)
print(test_losses)

[3.7965379161834716, 2.1489966821670534, 0.5881670632362366, 0.23717922800779342, 0.13997820371389388, 0.09943667326867581, 0.07771902143955231, 0.061377854824066165, 0.05012591248750687, 0.04223234912753105, 0.036214521542191506, 0.030954163778573275, 0.026210884910076858, 0.023867902413010598, 0.020569807728752495, 0.018399599589407446, 0.01579626458324492, 0.01360920101776719, 0.012616449125111104, 0.011075866144150496]
[3.0408279628753663, 1.050041361808777, 0.3059434094429016, 0.1581715569496155, 0.10778751868009567, 0.0812743023633957, 0.06967144238948822, 0.057519958436489105, 0.05249801413714886, 0.048249118618667125, 0.04558555785566568, 0.04222058665752411, 0.04033534550666809, 0.04333874249458313, 0.04080447668209672, 0.040461853824555874, 0.03995159102976322, 0.039922577600926164, 0.039960281191393734, 0.04017509805411101]


Let's test the model on a new sentence:

In [40]:
test_output = "ɒ d m i n i s t r aː t o r n ɒ k"
test_input = "adminisztrátornak"
print("input: " + test_input)
x = torch.tensor(encode_src(test_input)).unsqueeze(0).to(model.device)
y = torch.tensor(encode_tgt(test_output)).unsqueeze(0).to(model.device)
T = torch.tensor([x.shape[1]]).to(model.device)
U = torch.tensor([y.shape[1]]).to(model.device)
guess = model.greedy_search(x,T)[0]
print("truth: " + test_output)
print("guess: " + decode_tgt(guess))
print("")
y_guess = torch.tensor(guess).unsqueeze(0).to(model.device)
U_guess = torch.tensor(len(guess)).unsqueeze(0).to(model.device)

print("NLL of truth: " + str(model.compute_loss(x, y, T, U)))
print("NLL of guess: " + str(model.compute_loss(x, y_guess, T, U_guess)))

input: adminisztrátornak
truth: ɒ d m i n i s t r aː t o r n ɒ k
guess: ɒ d i n i s t r aː t o r n ɒ k

NLL of truth: tensor(0.0012, device='cuda:0', grad_fn=<TransducerBackward>)
NLL of guess: tensor(0.4555, device='cuda:0', grad_fn=<TransducerBackward>)


Observe that the negative log-likelihood of the guess is actually worse than that of the true label sequence (AKA, a "[search error](https://www.aclweb.org/anthology/D19-1331.pdf)"). This suggests that we could get better results using a beam search instead of the greedy search.